In [2]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to C:\Users\Jack
[nltk_data]     Ellis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv(r'..\..\..\data\clean_data\salaries_descriptions_locations.csv',encoding='utf-8')

In [4]:
stpwords = stopwords.words('english') + stopwords.words('french')
stpwords.append('')
print(stpwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# N-Grams

In [5]:
from nltk import ngrams

In [6]:
#This function returns all n grams as well as descriptions splits for a given dataset with Description as a column
def box_of_words(df,n):

    allDescriptions = []
    allGrams = []

    count = 0
    for description in df.Description: 
        sentences = description.split('\n')
        grams = []
        
        allTokens = []
        for sentence in sentences:
            tokens = sentence.split(' ')
            tokens_without_sw = [word for word in tokens if not word in stpwords]
            allTokens += tokens_without_sw
        
        finalTokens = []
        for i in range(len(allTokens)):
            cleanWord = re.sub('[\W_]+', '', allTokens[i])
            finalTokens.append(cleanWord)
        
        grams = list(ngrams(finalTokens, n))
        _ = ''
        
        grams = [gram for gram in grams if _ not in gram]
        
        allDescriptions.append(grams)
        allGrams += grams
        
    allGrams = list(dict.fromkeys(allGrams))
    
    return (allGrams,allDescriptions)

#This function gets the count for word sequences in a given dataset
def getGramCounts(grams,descriptions):
    allDescriptionsBOW = {i:0 for i in grams}
    
    for description in descriptions:
        for gram in description:
            allDescriptionsBOW[gram] += 1
    
    keys = list(allDescriptionsBOW.keys())
    values = list(allDescriptionsBOW.values())

    myList = []
    for i in range(len(keys)):
        myList.append([keys[i],values[i]])
    myList = sorted(myList,key=lambda l:l[1], reverse=True)
    
    return myList

#Create one-hot encoded variables for top numGrams appearing word sequences
def createIndicators(grams,descriptions,df,numGrams):
    
    for i in range(numGrams):
        
        gram = grams[i][0]
        
        newColoumValues = []
        for description in descriptions:
            if gram in description:
                newColoumValues.append(1)
            else:
                newColoumValues.append(0)
        
        df[gram] = newColoumValues

In [17]:
grams,descriptions = box_of_words(df,3)

In [18]:
gramCounts = getGramCounts(grams,descriptions)

In [19]:
gramCounts

[[('sexual', 'orientation', 'gender'), 261],
 [('race', 'color', 'religion'), 218],
 [('orientation', 'gender', 'identity'), 196],
 [('without', 'regard', 'race'), 193],
 [('regard', 'race', 'color'), 182],
 [('employment', 'without', 'regard'), 172],
 [('receive', 'consideration', 'employment'), 168],
 [('consideration', 'employment', 'without'), 168],
 [('equal', 'opportunity', 'employer'), 167],
 [('applicants', 'receive', 'consideration'), 160],
 [('qualified', 'applicants', 'receive'), 158],
 [('protected', 'veteran', 'status'), 158],
 [('All', 'qualified', 'applicants'), 145],
 [('natural', 'language', 'processing'), 134],
 [('gender', 'gender', 'identity'), 128],
 [('2', 'years', 'experience'), 125],
 [('color', 'national', 'origin'), 124],
 [('people', 'around', 'world'), 122],
 [('color', 'religion', 'sex'), 120],
 [('Senior', 'Data', 'Scientist'), 120],
 [('machine', 'learning', 'techniques'), 115],
 [('gender', 'identity', 'expression'), 114],
 [('machine', 'learning', 'mode

In [20]:
createIndicators(gramCounts,descriptions,df,20)

In [21]:
df

,Unnamed: 0,Unnamed: 0.1,Country,Description,Glassdoor Salary Estimate,Position_clean,Unnamed: 0.1.1,base_pay,bonus_pay,cash_bonus,...,"(qualified, applicants, receive)","(protected, veteran, status)","(All, qualified, applicants)","(natural, language, processing)","(gender, gender, identity)","(2, years, experience)","(color, national, origin)","(people, around, world)","(color, religion, sex)","(Senior, Data, Scientist)"
0,0,0,US,Facebook's mission is to give people the power...,193225.0,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0
1,1,1,US,Facebook's mission is to give people the power...,193225.0,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0
2,2,2,US,Facebook's mission is to give people the power...,184055.0,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0
3,3,3,US,Facebook's mission is to give people the power...,184055.0,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0
4,4,4,US,Facebook's mission is to give people the power...,184055.0,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0
5,5,5,US,Facebook's mission is to give people the power...,184055.0,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0
6,6,6,US,Facebook's mission is to give people the power...,185365.0,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0
7,7,7,US,Facebook's mission is to give people the power...,NaN,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0
8,8,8,US,Facebook's mission is to give people the power...,161785.0,Data Scientist,1,198153.22,19650.00,19650.00,...,0,1,0,0,1,1,1,1,0,0
9,9,9,US,Facebook's mission is to give people the power...,193225.0,Data Scientist,1190,198153.22,19650.00,19650.00,...,0,1,0,0,1,0,1,1,0,0


In [22]:
df.to_csv('../../../data/clean_data/salaries_descriptions_locations_gramcounts.csv')

# Language Analysis

This code was used to analyze the importance of dealing with french words or droppign them. It was found that there was very few strictly french descriptions that were droppable.

In [14]:
grams,descriptions = box_of_words(df,1)

In [15]:
from langdetect import detect
from langdetect import detect_langs

In [16]:
description = descriptions[3]
words = ''
for i in range(len(description)):
    words += ' ' + description[i][0]

In [17]:
count = 0
indexes = []
for description in descriptions:
    words = ''
    for i in range(len(description)):
        words += ' ' + description[i][0]
    langs = detect_langs(words)
    if len(langs)>1:
        indexes.append(count)
    count += 1
print(len(indexes))

28


In [18]:
count = 0
indexes = []
for description in descriptions:
    words = ''
    for i in range(len(description)):
        words += ' ' + description[i][0]
    lang = detect(words)
    if lang == 'fr':
        indexes.append(count)
    count += 1
print(len(indexes))

68


In [19]:
indexes

[109,
 195,
 248,
 257,
 295,
 306,
 307,
 314,
 315,
 318,
 319,
 320,
 322,
 325,
 326,
 329,
 332,
 334,
 369,
 389,
 394,
 396,
 400,
 629,
 633,
 636,
 642,
 644,
 660,
 667,
 668,
 669,
 670,
 695,
 696,
 697,
 698,
 705,
 707,
 708,
 715,
 726,
 741,
 758,
 764,
 772,
 790,
 793,
 794,
 801,
 821,
 823,
 831,
 931,
 949,
 958,
 988,
 1002,
 1009,
 1010,
 1017,
 1031,
 1035,
 1039,
 1040,
 1041,
 1073,
 1078]

In [47]:
dist = []
count = 0
for index in indexes:
    count += 1
    print(count)
    french = 0
    english = 0
    for word in descriptions[index]:
        if not word[0].isnumeric():
            lang = detect(word[0])
            if lang == 'en':
                english == 1
            elif lang == 'fr':
                french += 1
    dist.append([english,french])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68


In [48]:
dist

[[0, 281],
 [0, 108],
 [0, 257],
 [0, 281],
 [0, 153],
 [0, 111],
 [0, 266],
 [0, 8],
 [0, 168],
 [0, 212],
 [0, 104],
 [0, 183],
 [0, 107],
 [0, 100],
 [0, 170],
 [0, 45],
 [0, 189],
 [0, 259],
 [0, 268],
 [0, 277],
 [0, 208],
 [0, 264],
 [0, 247],
 [0, 282],
 [0, 8],
 [0, 139],
 [0, 267],
 [0, 208],
 [0, 252],
 [0, 268],
 [0, 171],
 [0, 165],
 [0, 252],
 [0, 107],
 [0, 213],
 [0, 109],
 [0, 422],
 [0, 257],
 [0, 185],
 [0, 106],
 [0, 103],
 [0, 43],
 [0, 216],
 [0, 112],
 [0, 257],
 [0, 236],
 [0, 123],
 [0, 135],
 [0, 267],
 [0, 272],
 [0, 201],
 [0, 182],
 [0, 180],
 [0, 115],
 [0, 231],
 [0, 139],
 [0, 269],
 [0, 248],
 [0, 261],
 [0, 319],
 [0, 188],
 [0, 187],
 [0, 110],
 [0, 112],
 [0, 319],
 [0, 111],
 [0, 152],
 [0, 181]]

In [31]:
df2 = df[df['index'].isin(indexes)]

In [32]:
df2

,index,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded
109,109,Balcan Plastics Inc,Les Plastiques Balcan compte parmi les chefs d...,NaN,NaN,Montreal,"Analyste de données, prévision de ventes / Sal...",NaN,NaN,NaN,NaN,NaN,NaN
195,195,adMare BioInnovations,About adMare BioInnovations:\r\r\r\r\r\r\r\r\n...,NaN,NaN,Montreal,Senior Scientist - Cell Biology,3.4,Less than $1 million (USD),NaN,Unknown,Company - Private,NaN
248,248,Thales,"Location: Quebec City, CanadaDans des marchés ...",NaN,Aerospace & Defense,Quebec,Optimization Operational Research Scientist,3.8,$10+ billion (USD),Aerospace & Defense,10000+ Employees,Company - Public,2000.0
257,257,Square Enix Montréal,// English version will follow...//\r\r\r\r\r\...,NaN,Video Games,Montreal,Analyste de données en chef / Lead Data Analyst,3.5,Unknown / Non-Applicable,Media,1001 to 5000 Employees,Company - Public,1975.0
295,295,agileDSS,Nous sommes à la recherche d’un(e) Data Engine...,NaN,IT Services,Montreal,Data Engineer / Data Analyst,4.4,$10 to $25 million (USD),Information Technology,51 to 200 Employees,Company - Private,2005.0
306,306,Ardene,WHO WE ARE:\r\r\r\r\nArdene is the ultimate va...,NaN,"Department, Clothing, & Shoe Stores",Saint-Laurent,Digital Data Analyst / Analyste de données num...,3.2,Unknown / Non-Applicable,Retail,1001 to 5000 Employees,Company - Private,1982.0
307,307,ODESIA,Description\r\r\r\r\nRôle d'analyste de donnée...,NaN,Consulting,Montreal,Data Analyst - BI Environment,5.0,$5 to $10 million (USD),Business Services,51 to 200 Employees,Company - Private,NaN
314,314,ZeniMax Media Inc.,Your Privacy:\r\r\r\r\nZeniMax understands the...,NaN,Video Games,Montreal,Spécialiste en Analyse des données / Data Analyst,3.8,Unknown / Non-Applicable,Media,201 to 500 Employees,Company - Private,1999.0
315,315,PLB International inc,"At PLB, the Data Analyst and Demand Planning, ...",NaN,Consumer Products Manufacturing,Boucherville,"Demand Planner, Data Analyst",3.9,$25 to $50 million (USD),Manufacturing,51 to 200 Employees,Company - Private,1969.0
318,318,Square Enix Montreal,// ENGLISH VERSION WILL FOLLOW ...//\r\r\r\r\n...,NaN,Video Games,Montreal,Analyste de données / Data Analyst,3.5,Unknown / Non-Applicable,Media,1001 to 5000 Employees,Company - Public,1975.0


In [33]:
df3 = pd.read_csv('../../../data/clean_data/salaries_ca.csv')

,Unnamed: 0,company,position,salary,lower_bound_salary,upper_bound_salary,base_pay,bonus_pay,cash_bonus,stock_bonus
0,0,Capital One,Data Scientist,76883,73000,125000,76883.0,5885.0,5885.0,3272.0
1,1,RBC,Data Scientist,90000,55000,135000,90000.0,12589.0,11983.0,3147.0
2,2,Scotiabank,Data Scientist,96176,74000,103000,96176.0,10051.0,10051.0,3327.0
3,3,TD,Data Scientist,79473,50000,129000,79473.0,6062.0,6062.0,2393.0
4,4,Banque Nationale du Canada/National Bank of Ca...,Data Scientist,74945,71000,90000,74945.0,5045.0,5045.0,1536.0
5,5,Loblaw Companies,Data Scientist,90000,12000,107000,90000.0,9000.0,10008.0,5000.0
6,6,Shopify,Data Scientist,95489,85000,115000,95489.0,NaN,NaN,32840.0
7,7,Bell Canada,Data Scientist,80501,66000,108000,80501.0,11442.0,11442.0,NaN
8,8,RBC,Data Scientist - Hourly Intern,56160,43680,62400,NaN,NaN,NaN,NaN
9,9,IBM,Data Scientist,85187,71000,87000,85187.0,509.0,509.0,NaN


In [35]:
finaldf = pd.merge(df2, df3, left_on='Company', right_on='company')

In [36]:
finaldf

,index,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,...,Unnamed: 0,company,position,salary,lower_bound_salary,upper_bound_salary,base_pay,bonus_pay,cash_bonus,stock_bonus
0,793,Boeing,Senior Software Solution Developer\r\r\r\r\r\r...,NaN,Aerospace & Defense,Montreal,Senior Software Solution Developer,3.7,$10+ billion (USD),Aerospace & Defense,...,12,Boeing,Data Scientist,100253,74000,108000,100253.0,3147.0,3147.0,NaN
1,794,Boeing,Senior DevOps SysOps Specialist\r\r\r\r\r\r\r\...,NaN,Aerospace & Defense,Montreal,Senior DevOps SysOps Specialist,3.7,$10+ billion (USD),Aerospace & Defense,...,12,Boeing,Data Scientist,100253,74000,108000,100253.0,3147.0,3147.0,NaN
